# Skill estimation using Stan

In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(66)
import pickle

## Model that defined skill's distribution with more features

In [2]:
skill_model = """
data {
  int<lower=1> N;             // Total number of players
  int<lower=1> E;             // number of games
  real<lower=0> scale;        // scale value for probability computation
  int<lower=0,upper=1> win[E]; // PA wins vs PB
  int PA[E];                  // player info between each game
  int PB[E];                  // 
  real scores[E];              // scores for each game
}
parameters {
  vector [N] skill;           // skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
    scores[i] ~ gamma( fabs(skill[PA[i]] - skill[PB[i]]) + 2, 0.5 );
  }   // win probability is a logit function of skill difference
}
"""

Now, compile the model.  

In [3]:
sm = pystan.StanModel(model_code = skill_model)

In [4]:
with open('skill_model_scores_diff.pkl', 'wb') as f: pickle.dump(sm, f)

In [5]:
# try:     # load it if already compiled
#     sm = pickle.load(open('skill_model_scores.pkl', 'rb'))
# except:  # ow, compile and save compiled model
#     sm = pystan.StanModel(model_code = skill_model)
#     with open('skill_model_scores.pkl', 'wb') as f: pickle.dump(sm, f)

## Processing data

In [6]:
def load_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    
    # Sparsifying parameters (discard some training examples):
    # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    wins = []
    playerA, playerB = [], []
    raceA, raceB = [], []
    nplayers = len(playerid)
    nplays = np.zeros( (nplayers,nplayers) )
    race = {'P':0, 'T':1, 'Z':2}
    scoresA = []
    scoresB = []
    scores = []

    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10 or csv[6] == 'R' or csv[7] == 'R':
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                
                playerA.append(a+1)
                playerB.append(b+1)
                wins.append(1 if aw else 0) 
                raceA.append(race[csv[6]])
                raceB.append(race[csv[7]])
                sa, sb = csv[3].split('–')
                scores.append(abs(int(sa)-int(sb)))
#                 scores.append((int(sa)-int(sb)))
                scoresA.append(int(sa))
                scoresB.append(int(sb))

    return playerid,nplayers,playerA,playerB,raceA, raceB, wins, scores

In [7]:
playerid,nplayers,playerA,playerB,raceA, raceB, wins, scores = load_data()

In [8]:
print('summary: ')
print('# players', nplayers)
print('# games', len(wins))
print('player A', playerA[:10])
print('player B', playerB[:10])
print('wins', wins[:10])
print('scores', scores[:10])

summary: 
# players 999
# games 4678
player A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
player B [2, 3, 4, 5, 6, 7, 8, 9, 10, 9]
wins [0, 0, 0, 0, 1, 1, 1, 0, 0, 1]
scores [2, 1, 2, 1, 1, 1, 1, 2, 2, 1]


We also need the observed data: number of players and games, which pairs played each game, and who won:

In [9]:
skill_data = {
    'N': nplayers,
    'E': len(wins),
    'scale': 0.4,
    'win':wins,
    'PA': playerA,
    'PB': playerB,
    'scores': scores
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [10]:
fit = sm.sampling(data=skill_data, iter=2000, chains=4, n_jobs = 4, verbose = True)

In [11]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [12]:
samples

OrderedDict([('skill',
              array([[-0.01410025, -0.22270211,  0.08216647, ..., -0.25304753,
                      -0.8986737 , -0.58046078],
                     [ 0.18652951,  0.04970448,  0.17656142, ...,  0.53103764,
                       1.05356926,  0.20388716],
                     [ 0.26069781,  0.34515435,  0.42268362, ...,  0.26069943,
                      -0.2814254 , -0.19252609],
                     ...,
                     [ 0.07332413,  0.1192781 ,  0.21281536, ...,  0.02751995,
                      -0.03330764,  0.09360843],
                     [-0.04506887, -0.32122863, -0.0713199 , ..., -0.35967077,
                       0.74461244, -0.05583605],
                     [ 0.05827586, -0.04482001,  0.16566944, ..., -0.60075256,
                       0.03987125, -0.18841636]])),
             ('lp__',
              array([-8634.39650095, -8658.82599988, -8621.43229158, ...,
                     -8658.70307676, -8622.33324285, -8618.4230522 ]))])

In [13]:
samples['skill'].shape # 2*100 iterations? 999 players

(4000, 999)

Remember to save the prediction!

In [14]:
with open('skill_hat_scores_diff.pkl', 'wb') as f: 
    pickle.dump(samples, f)

## Sample Model Evaluation

In [15]:
skill_hat = pickle.load(open('skill_hat_scores_diff.pkl', 'rb'))

In [16]:
skill_hat = skill_hat['skill']

In [17]:
def load_valid_data(playerid, dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='valid'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')


  # Sparsifying parameters (discard some training examples):
  # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
  # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
  # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)
    
    games = []
    race = {'P':0, 'T':1, 'Z':2}
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10 or csv[6] == 'R' or csv[7] == 'R':
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                nwins[a,b] += aw
                nwins[b,a] += bw
                games.append((a,b,aw,race[csv[6]],race[csv[7]]))
    
    return nplayers, nplays, nwins, games

In [18]:
nplayers_val, nplays_val, nwins_val, games_val = load_valid_data(playerid)

In [19]:
print('summary: ', nplayers_val)
print(nplays_val.shape, nplays_val.sum())
print(nwins_val.shape, nwins_val.sum())
print('games', len(games_val))

summary:  999
(999, 999) 9542.0
(999, 999) 4788.0
games 4771


In [20]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skill, nplayers, nplays, nwins, games):
    
    loss = 0.
    binary_loss = 0.
    for i in range(nplayers):
        for j in range(i+1, nplayers):
            if nplays[i, j] == 0:
                continue
            prob = nwins[i,j] / nplays[i,j]
            prob_hat = logit( skill_data['scale']*(skill[:,i]-skill[:,j]) ).mean()
            loss += np.abs(prob_hat - prob)
            binary_loss += np.logical_xor(prob_hat >= 0.5, prob >= 0.5)
    
    loss /= (nplays > 0).sum()/2
    binary_loss /= (nplays > 0).sum()/2
    
    return loss, binary_loss


In [21]:
loss = prediction_loss(skill_hat, nplayers_val, nplays_val, nwins_val, games_val)

In [22]:
loss

(0.45426590510171844, 0.18932527693857)

In [25]:
def loss_per_game(skill, games):
    
    loss = 0.

    for game in games:
        a, b, w, ra,rb = game
        prob_hat = logit( skill_data['scale']*(skill[:,a]-skill[:,b]) ).mean()
        loss += np.logical_xor(prob_hat >= 0.5, w)

    loss /= len(games)
    
    return loss


In [26]:
loss_per_game(skill_hat, games_val)

0.22783483546426325